In [199]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random
import pickle
from PIL import Image

%matplotlib inline

In [2]:
# image folder needs to be in current word directory (cwd)

cwd = os.getcwd()

object_types = os.listdir(cwd + "/Dataset_downsized/")
object_types = list(filter(lambda s: s[0] != '.', object_types))

pictures = []
labels   = []

for obj in object_types:
    
    file_names = os.listdir(cwd + "/Dataset_downsized/" + obj)
    
    for pic in file_names:
        pictures.append(mpimg.imread(cwd + "/Dataset_downsized/" + obj + "/" + pic))
        labels.append(object_types.index(obj))
    print("Finished loading all " + obj + " pictures, number of pictures: " + str(len(file_names)))

print("\nFinished loading all images")
print("Total number of images: " + str(len(pictures)))

assert len(pictures) == len(labels)

Finished loading all cardboard pictures, number of pictures: 403
Finished loading all glass pictures, number of pictures: 501
Finished loading all metal pictures, number of pictures: 410
Finished loading all paper pictures, number of pictures: 594
Finished loading all plastic pictures, number of pictures: 482
Finished loading all trash pictures, number of pictures: 137

Finished loading all images
Total number of images: 2527


In [3]:
pictures = np.array(pictures)
labels   = np.array(labels)

In [4]:
def split_data(x_data, labels, ratio):
    assert ratio > 0.5 and ratio < 1
    
    N = int(ratio*len(x_data))
    data_with_labels = list(zip(x_data, labels))
    
    random.shuffle(data_with_labels)
    
    shuffled_data, shuffled_labels = zip(*data_with_labels)
    
    x_train = np.array(shuffled_data[:N])
    y_train = np.array(shuffled_labels[:N])
    x_test = np.array(shuffled_data[N:])
    y_test = np.array(shuffled_labels[N:])
    
    return x_train, y_train, x_test, y_test

In [5]:
def max_at_index(index, max_val):
    zero_array = np.zeros(max_val+1)
    zero_array[index] = 1
    
    return zero_array


def reshape_data(x_train, y_train, x_test, y_test):
    num_classes = max(y_train)
    n_train, num_rows, num_cols, three = np.shape(x_train)
    n_test, _, _, _ = np.shape(x_test)
    
    x_train = x_train.astype("float32")/255
    x_test  = x_test.astype("float32")/255
    
    assert three == 3
    
    if K.image_data_format() == 'channels_first':
        x_train = x_train.reshape(n_train, three, num_rows, num_cols)
        x_test = x_test.reshape(n_test, three, num_rows, num_cols)
        input_shape = (three, img_rows, img_cols)
        
    else:
        x_train = x_train.reshape(n_train, num_rows, num_cols, three)
        x_test = x_test.reshape(n_test, num_rows, num_cols, three)
        input_shape = (num_rows, num_cols, 3)
        
    y_train = np.array([max_at_index(index, num_classes) for index in y_train])
    y_test  = np.array([max_at_index(index, num_classes) for index in y_test])
    
    return x_train, y_train, x_test, y_test

In [6]:
x_train, y_train, x_test, y_test = split_data(pictures, labels, 0.8)
print(np.shape(x_train[0]))

(64, 48, 3)
[[[207 209 221]
  [207 209 221]
  [208 210 222]
  ..., 
  [163 163 173]
  [160 160 170]
  [157 157 167]]

 [[207 209 221]
  [208 210 222]
  [208 210 222]
  ..., 
  [162 162 172]
  [158 158 168]
  [155 155 165]]

 [[207 209 221]
  [208 210 222]
  [208 210 222]
  ..., 
  [159 159 169]
  [156 156 166]
  [153 153 163]]

 ..., 
 [[209 211 224]
  [209 211 224]
  [209 211 224]
  ..., 
  [168 170 183]
  [166 168 181]
  [164 166 179]]

 [[209 211 224]
  [209 211 224]
  [209 211 224]
  ..., 
  [170 172 185]
  [167 169 182]
  [165 167 180]]

 [[209 211 224]
  [209 211 224]
  [209 211 224]
  ..., 
  [170 172 185]
  [168 170 183]
  [166 168 181]]]


In [7]:
x_train, y_train, x_test, y_test = reshape_data(x_train, y_train, x_test, y_test)
print(np.shape(x_train[0]))

(64, 48, 3)
[[[ 0.81176472  0.81960785  0.86666667]
  [ 0.81176472  0.81960785  0.86666667]
  [ 0.81568629  0.82352942  0.87058824]
  ..., 
  [ 0.63921571  0.63921571  0.67843139]
  [ 0.627451    0.627451    0.66666669]
  [ 0.6156863   0.6156863   0.65490198]]

 [[ 0.81176472  0.81960785  0.86666667]
  [ 0.81568629  0.82352942  0.87058824]
  [ 0.81568629  0.82352942  0.87058824]
  ..., 
  [ 0.63529414  0.63529414  0.67450982]
  [ 0.61960787  0.61960787  0.65882355]
  [ 0.60784316  0.60784316  0.64705884]]

 [[ 0.81176472  0.81960785  0.86666667]
  [ 0.81568629  0.82352942  0.87058824]
  [ 0.81568629  0.82352942  0.87058824]
  ..., 
  [ 0.62352943  0.62352943  0.66274512]
  [ 0.61176473  0.61176473  0.65098041]
  [ 0.60000002  0.60000002  0.63921571]]

 ..., 
 [[ 0.81960785  0.82745099  0.87843138]
  [ 0.81960785  0.82745099  0.87843138]
  [ 0.81960785  0.82745099  0.87843138]
  ..., 
  [ 0.65882355  0.66666669  0.71764708]
  [ 0.65098041  0.65882355  0.70980394]
  [ 0.64313728  0.65098

The data is ready for the convolutional neural network!

In [8]:
def find_max(array):
    return list(array).index(max(array))

def keras_cnn(x_train, y_train, x_test, y_test, batch_size):
    
    _, num_rows, num_cols, three = np.shape(x_train)
    input_shape = (num_rows, num_cols, three)
    num_classes = len(y_train[0])
    epochs      = 15
    
    assert all(len(y) == len(y_train[0]) for y in y_train)
    
    positional_cnn = {}
    

        
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss="categorical_crossentropy",
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs)

    # score = model.evaluate(x_test, y_test[:, c], verbose=0)
    
    print("Training done!")
    
    return model

In [9]:
print("Size of x_train: ", np.shape(x_train))
print("Size of y_train: ", np.shape(y_train))
print("Size of x_test: ", np.shape(x_test))
print("Size of y_test: ", np.shape(y_test))

Size of x_train:  (2021, 64, 48, 3)
Size of y_train:  (2021, 6)
Size of x_test:  (506, 64, 48, 3)
Size of y_test:  (506, 6)


In [10]:
cnn = keras_cnn(x_train, y_train, x_test, y_test, 64)

Epoch 1/8
2021/2021 [==============================] - 31s 15ms/step - loss: 1.8811 - acc: 0.2128

In [11]:
def compute_accuracy(predictor, x_test, y_test):

    prediction       = np.array([find_max(array) for array in predictor.predict(x_test)])
    predicted_labels = np.array([find_max(array) for array in y_test])
    accuracy         = np.mean(prediction == predicted_labels)
    
    return accuracy

In [14]:
accuracy = compute_accuracy(cnn, x_test, y_test)
print("Final Accuarcy: {}%".format(accuracy*100))

Final Accuarcy: 50.790513833992094%


In [206]:
cnn.save("final_cnn.h5")

In [204]:
type(cnn)

keras.models.Sequential

## Everything below is an analysis of the predictor

In [45]:
pred_labels = {"test": cnn.predict(x_test),
               "train": cnn.predict(x_train)}

In [154]:
def simple_neural_network(pred_label_train, true_label_train, pred_label_test, true_label_test, batch_size):
    '''
    • batch_size: feed data in batches
    • 1 hidden layer
    '''
    n, num_class = np.shape(pred_label_train)
    NUM_EPOCHS   = 1000
    NUM_HN       = 10
    
    assert np.shape(pred_label_train) == np.shape(true_label_train)
    
    x = tf.placeholder(tf.float32, shape=[None, num_class])
    y = tf.placeholder(tf.float32, shape=[None, num_class])

    batch_indices = [b*batch_size for b in range(int(n/batch_size)+1)] + [n]
    batch_tup     = list(zip(batch_indices[:-1], batch_indices[1:]))

    weights = {"hidden": tf.Variable(tf.random_normal([num_class, NUM_HN])),
               "output": tf.Variable(tf.random_normal([NUM_HN, num_class]))}
    bias    = {"hidden": tf.Variable(tf.random_normal([NUM_HN])),
               "output": tf.Variable(tf.random_normal([num_class]))}
               
    hidden_layer = tf.nn.sigmoid(tf.add(tf.matmul(x, weights["hidden"]), bias["hidden"]))
    prediction   = tf.add(tf.matmul(hidden_layer, weights["output"]), bias["output"])
    cost         = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
    optimizer    = tf.train.GradientDescentOptimizer(0.01).minimize(cost)
    output       = []
    
    with tf.Session() as sess:        
        sess.run(tf.global_variables_initializer())
        
        pred_labels  = {}
        train_labels = transform_back(true_label_train)
        accuracy     = 0
        
        for epoch in range(NUM_EPOCHS+1):    
            
            for b in batch_tup:
                # e.g.: b = (128, 160)
                sess.run([optimizer, cost], feed_dict={x: pred_label_train[b[0]:b[1]], y: true_label_train[b[0]:b[1]]})
            '''
            feed training data, compute testing data for accuracy
            '''
            output    = prediction.eval({x:pred_label_test})
            predicted = np.array([find_max(array) for array in output])
            accuracy  = np.mean(predicted == np.array([find_max(array) for array in true_label_test]))
            
            if epoch % 100 == 0:
                print("Epoch {}, accuracy = {}%".format(epoch, accuracy*100))
                    
        print("\nFinal Accuracy: {}%".format(accuracy*100))
        
    
    return output


def transform_back(labels):
    return np.array([np.argmax(label) for label in labels])

In [157]:
adjusted_predictions = simple_neural_network(pred_labels["train"], y_train, pred_labels["test"], y_test, 32)

Epoch 0, accuracy = 21.541501976284586%
Epoch 100, accuracy = 55.92885375494071%
Epoch 200, accuracy = 54.1501976284585%
Epoch 300, accuracy = 54.347826086956516%
Epoch 400, accuracy = 54.1501976284585%
Epoch 500, accuracy = 55.533596837944664%
Epoch 600, accuracy = 55.73122529644269%
Epoch 700, accuracy = 56.7193675889328%
Epoch 800, accuracy = 56.7193675889328%
Epoch 900, accuracy = 56.7193675889328%
Epoch 1000, accuracy = 56.52173913043478%

Final Accuracy: 56.52173913043478%


In [190]:
from tabulate import tabulate

final_labels = {"true": np.array([find_max(a) for a in y_test]),
                "pre": np.array([find_max(a) for a in pred_labels["test"]]),
                "post": np.array([find_max(a) for a in adjusted_predictions])}


ModuleNotFoundError: No module named 'tabulate'